In [3]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
import mysql.connector

# inisialisasi array untuk menampilkan hasil
id_detail = []
keterangan_lowongan = []
deskripsi_lowongan = []
stam_detail = []

# konektor ke database
db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='UnNgGrape'
        )
cursor = db.cursor()

cursor.execute("SELECT id_karir, link_lowongan FROM karir")
raw_karir = cursor.fetchall()
for row in raw_karir:
    id_karir = row[0]
    link = row[1]

    r = Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
    response = urlopen(r).read()
    soup = BeautifulSoup(response, "lxml")

    raw_detail = soup.find_all("div","b-opportunity-info__main")
    for p in raw_detail:

        # menggunakan try except karena ada beberapa estimasi keterangan yang tidak terlampir
        try:
            
            keterangan = p.find("div", "b-opportunity-info__main__left" ).get_text(separator=". ")
        except:
            keterangan = "-"
        
        # menggunakan try except karena ada beberapa deskripsi yang NoneType
        try:
            raw_deskripsi = p.find("div", "html-content b-opportunity-show__box" ).get_text(separator=". ")
            nltk_tokens = nltk.sent_tokenize(raw_deskripsi)
            deskripsi = ""

            for x in nltk_tokens:
                if x == "Disclaimer:.":
                    break
                deskripsi = deskripsi + " " + x
        except:
            deskripsi = "Eror: deskripsi Nonetype"

        raw_stam = keterangan + " " + deskripsi
        # create stemmer
        # nltk
        ps = PorterStemmer()
        # sastrawi
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        # stemming process
        # nltk
        words = word_tokenize(raw_stam)
        nltk_stammer = ""
        for w in words:
            nltk_stammer = nltk_stammer + " " + ps.stem(w)
        # sastrawi
        output_stammer = stemmer.stem(nltk_stammer)

        # mengecek apakah data sudah adat atau belum
        cursor.execute("SELECT id_karir FROM detail_karir WHERE id_karir = '" + str(id_karir) + "'")
        data = cursor.fetchall()
        # if else untuk pengecekan input data
        if data:
            print("Data exists")
        else:
            cursor.execute(
                        "INSERT INTO detail_karir(id_karir, keterangan_lowongan, deskripsi_lowongan, stem_detail)"
                        "VALUES ('"+ str(id_karir) +"', '"+ keterangan.replace("'", "").replace('"', '') +"', '"+ deskripsi.replace("'", "").replace('"', '') +"', '"+ output_stammer +"')"
                        )
            
            db.commit()
            print("Data has been added")

        id_detail.append(id_karir)
        keterangan_lowongan.append(keterangan)
        deskripsi_lowongan.append(deskripsi)
        stam_detail.append(output_stammer)

detail_dict ={'id':id_detail, 'keterangan':keterangan_lowongan, 'deskripsi':deskripsi_lowongan, 'stam':stam_detail}
df = pd.DataFrame(detail_dict, columns = ['id', 'keterangan', 'deskripsi', 'stam'])

df.sort_values('id', ascending=True)

Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data exists
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been ad

,id,keterangan,deskripsi,stam
0,257,"Fungsi Kerja. Teknik, Sipil, Teknik, Mesin. Je...",Persyaratan. Lulusan minimal S1 dari Jurusan ...,fungsi kerja teknik sipil teknik mesin jenjang...
1,261,Fungsi Kerja. Administrasi. Jenjang Karir. Pem...,"Persyaratan. Wanita, Usia Max 25th. Pendidika...",fungsi kerja administrasi jenjang karir mula s...
2,267,"Fungsi Kerja. Sumber Daya Manusia, Pelatih, Pe...",Persyaratan. Male/Female between 30 until 37 ...,fungsi kerja sumber daya manusia latih didik j...
3,272,Fungsi Kerja. Penjualan dan Pemasaran. Jenjang...,Persyaratan. 1. Komitmen\r\n. 2. Mau belajar ...,fungsi kerja jual dan pasar jenjang karir dire...
4,273,Fungsi Kerja. Pengembangan Bisnis. Jenjang Kar...,Persyaratan. • Usia maksimal 30 tahun. • Dipl...,fungsi kerja kembang bisni jenjang karir mula ...
...,...,...,...,...
598,953,Fungsi Kerja. Layanan Pelanggan. Jenjang Karir...,Persyaratan. Kualifikasi :. - Pria/Wanita. - ...,fungsi kerja layan langgan jenjang karir mula ...
599,954,"Fungsi Kerja. TI, Programmer. Jenjang Karir. S...",Persyaratan. Menguasa PHP terutama dala pemro...,fungsi kerja ti programm jenjang karir staf se...
600,955,"Fungsi Kerja. Pemasaran (Non-Teknis), Penjuala...",Persyaratan. 1. Pendidikan Sarjana (S1). 2. U...,fungsi kerja pasar non-tekni jual non-tekni je...
601,956,Fungsi Kerja. Management Trainee. Jenjang Kari...,Persyaratan. - Pendidikan S1. - Usia 25 - 35t...,fungsi kerja manag traine jenjang karir manaj ...
